# Notebook for placing the hashed trajectories into buckets

In [5]:
# Importing nescessary modules
import os
import sys
import shutil
import timeit as ti
from tqdm import tqdm

from multiprocessing import Pool

def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

# Example usage
project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")

from utils.helpers.save_trajectory import save_trajectory_hashes
from utils.helpers import file_handler as fh
from utils.helpers import metafile_handler as mfh
from schemes.lsh_disk import DiskLSH

Project root found: c:\Users\eivin\dev\JoonEndreLSH\masteroppgave


# GRID

## Using CityHash

In [ ]:
import json
import cityhash
import os
NUMBER_OF_TRAJECTORIES = 3050

# Paths
ROME_HASHED_TRAJECTORIES_OUTPUT_FOLDER = "../../dataset/hashed_data/grid/rome/"
ROME_HASHED_TRAJECTORIES_FOLDER_META_FILE = f"{ROME_HASHED_TRAJECTORIES_OUTPUT_FOLDER}META-{NUMBER_OF_TRAJECTORIES}.txt"

ROME_FULL_TRAJECTORIES_OUTPUT_FOLDER = "../../dataset/rome/output/"

# Dictionary for the bucket system
bucket_system = {}

# Get filenames from the metafile
files = mfh.read_meta_file(ROME_HASHED_TRAJECTORIES_FOLDER_META_FILE)

# Iterate through trajectory files and read their hashes
for filename in files:
    file_path = os.path.join(ROME_HASHED_TRAJECTORIES_OUTPUT_FOLDER, filename)
    
    # Read the hashes for the trajectory
    trajectory_hashes = fh.read_hash_file(file_path)
    
    # Iterate over each layer's hash
    for layer_hash in trajectory_hashes:
        # Convert the list of coordinates into a string
        hash_string = "_".join(map(str, layer_hash))
        
        # Use CityHash for creating a unique key
        hash_key = cityhash.CityHash128(hash_string)
        
        # Place trajectory into the appropriate bucket
        if hash_key not in bucket_system:
            bucket_system[hash_key] = []
        bucket_system[hash_key].append(filename)


# Write the bucket system to a JSON file
output_file_path = f"../../results_hashed/buckets/grid/rome/rome_{NUMBER_OF_TRAJECTORIES}.json"
with open(output_file_path, "w") as f:
    json.dump(bucket_system, f)

# Analyze and display results
total_buckets = len(bucket_system)
buckets_with_multiple = sum(1 for trajectories in bucket_system.values() if len(trajectories) > 1)
buckets_with_single = total_buckets - buckets_with_multiple
largest_bucket_size = max(len(trajectories) for trajectories in bucket_system.values())
largest_bucket = max(bucket_system, key=lambda key: len(bucket_system[key]))
print(f"Largest Bucket: {largest_bucket}")

print(f"Total Buckets: {total_buckets}")
print(f"Buckets with more than one trajectory: {buckets_with_multiple}")
print(f"Buckets with only one trajectory: {buckets_with_single}")
print(f"Largest Bucket Size: {largest_bucket_size}")

# Optional: Display distribution percentages
multiple_bucket_percentage = (buckets_with_multiple / total_buckets) * 100 if total_buckets > 0 else 0
single_bucket_percentage = (buckets_with_single / total_buckets) * 100 if total_buckets > 0 else 0

print(f"Percentage of buckets with more than one trajectory: {multiple_bucket_percentage:.2f}%")
print(f"Percentage of buckets with only one trajectory: {single_bucket_percentage:.2f}%")




Largest Bucket: 18335119550780238394043628368761195560
Total Buckets: 5
Buckets with more than one trajectory: 5
Buckets with only one trajectory: 0
Largest Bucket Size: 3050
Percentage of buckets with more than one trajectory: 100.00%
Percentage of buckets with only one trajectory: 0.00%


## Finds the largest bucket and outputs stats from this bucket

In [ ]:
import pandas as pd
from itertools import combinations

ROME_TRUE_SIMILARITY_FILE = "../../results_true/similarity_values/rome/dtw/rome-dtw-3050.csv"

# Load the similarity matrix CSV
similarity_df = pd.read_csv(ROME_TRUE_SIMILARITY_FILE, index_col=0)

# Results storage for statistics
bucket_stats = {}

# Filter buckets to process only those with more than one trajectory
filtered_buckets = {bucket: trajectories for bucket, trajectories in bucket_system.items() if len(trajectories) > 1}

# Track the bucket with the most trajectories
max_bucket_size = 0
max_bucket = None

# Process each filtered bucket
for bucket, trajectories in filtered_buckets.items():
    similarities = []
    best_pair = None
    worst_pair = None

    # Compute all pairwise similarities within the bucket
    for t1, t2 in combinations(trajectories, 2):
        # Strip `.txt` from the trajectory names
        t1_clean = t1.replace('.txt', '')
        t2_clean = t2.replace('.txt', '')

        # Ensure correct row-column order (t1_clean should be lexicographically larger)
        if t1_clean < t2_clean:
            t1_clean, t2_clean = t2_clean, t1_clean  # Swap for correct matrix access

        if t1_clean in similarity_df.index and t2_clean in similarity_df.columns:
            similarity = float(similarity_df.at[t1_clean, t2_clean])  # Convert to native Python float
            similarities.append(similarity)
            
            # Track the pair with the best similarity (lowest value)
            if best_pair is None or similarity < best_pair[0]:
                best_pair = (similarity, t1, t2)

            # Track the pair with the worst similarity (highest value)
            if worst_pair is None or similarity > worst_pair[0]:
                worst_pair = (similarity, t1, t2)

    if similarities:
        best_similarity = min(similarities)
        worst_similarity = max(similarities)
        avg_similarity = sum(similarities) / len(similarities)
        bucket_stats[bucket] = {
            "best": best_similarity,
            "worst": worst_similarity,
            "average": avg_similarity,
            "best_pair": best_pair,
            "worst_pair": worst_pair,
        }
        
        # Check if this bucket has the most trajectories
        if len(trajectories) > max_bucket_size:
            max_bucket_size = len(trajectories)
            max_bucket = bucket
    else:
        # Handle missing trajectory pairs in the CSV
        bucket_stats[bucket] = {"best": None, "worst": None, "average": None}

# Display results for the bucket with the most trajectories
if max_bucket:
    print(f"Bucket with the most trajectories: {max_bucket} ({max_bucket_size} trajectories)")
    best_similarity, best_t1, best_t2 = bucket_stats[max_bucket]["best_pair"]
    worst_similarity, worst_t1, worst_t2 = bucket_stats[max_bucket]["worst_pair"]
    print(f"Most Similar Pair (Lowest similarity value): {best_similarity} between {best_t1} and {best_t2}")
    print(f"Least Similar Pair (Highest similarity value): {worst_similarity} between {worst_t1} and {worst_t2}")
    print(f"Average Similarity inside bucket '{max_bucket}': {bucket_stats[max_bucket]['average']:.2f}")
else:
    print("No buckets with more than one trajectory found.")

# DISK

## Using CityHash

In [3]:
import cityhash
import os
import json

NUMBER_OF_TRAJECTORIES = 3050

# Paths
ROME_HASHED_TRAJECTORIES_OUTPUT_FOLDER = "../../dataset/hashed_data/disk/rome/"
ROME_HASHED_TRAJECTORIES_FOLDER_META_FILE = f"{ROME_HASHED_TRAJECTORIES_OUTPUT_FOLDER}META-{NUMBER_OF_TRAJECTORIES}.txt"

ROME_FULL_TRAJECTORIES_OUTPUT_FOLDER = "../../dataset/rome/output/"

# Dictionary for the bucket system
bucket_system = {}

# Get filenames from the metafile
files = mfh.read_meta_file(ROME_HASHED_TRAJECTORIES_FOLDER_META_FILE)

# Iterate through trajectory files and read their hashes
for filename in files:
    file_path = os.path.join(ROME_HASHED_TRAJECTORIES_OUTPUT_FOLDER, filename)
    
    # Read the hashes for the trajectory
    trajectory_hashes = fh.read_hash_file(file_path)
    
    # Iterate over each layer's hash
    for layer_hash in trajectory_hashes:

        if layer_hash == ['']:
            continue
            

        # Convert the list of coordinates into a string
        hash_string = "".join(map(str, layer_hash))


        
        # Use CityHash for creating a unique key
        hash_key = cityhash.CityHash128(hash_string)
        if hash_key == "":
            print(filename, hash_string)
        
        # Place trajectory into the appropriate bucket
        if hash_key not in bucket_system:
            bucket_system[hash_key] = []
        bucket_system[hash_key].append(filename)
    
# Write the bucket system to a JSON file
output_file_path = f"../../results_hashed/buckets/disk/rome/rome_{NUMBER_OF_TRAJECTORIES}.json"
with open(output_file_path, "w") as f:
    json.dump(bucket_system, f)

   
# Analyze and display results

total_buckets = len(bucket_system)
buckets_with_multiple = sum(1 for trajectories in bucket_system.values() if len(trajectories) > 1)
buckets_with_single = total_buckets - buckets_with_multiple
largest_bucket_size = max(len(trajectories) for trajectories in bucket_system.values())
largest_bucket = max(bucket_system, key=lambda key: len(bucket_system[key]))
print(f"Largest Bucket: {largest_bucket}")

print(f"Total Buckets: {total_buckets}")
print(f"Buckets with more than one trajectory: {buckets_with_multiple}")
print(f"Buckets with only one trajectory: {buckets_with_single}")
print(f"Largest Bucket Size: {largest_bucket_size}")

# Optional: Display distribution percentages
multiple_bucket_percentage = (buckets_with_multiple / total_buckets) * 100 if total_buckets > 0 else 0
single_bucket_percentage = (buckets_with_single / total_buckets) * 100 if total_buckets > 0 else 0

print(f"Percentage of buckets with more than one trajectory: {multiple_bucket_percentage:.2f}%")
print(f"Percentage of buckets with only one trajectory: {single_bucket_percentage:.2f}%")




Largest Bucket: 292977397802616694193368700077375428004
Total Buckets: 16355
Buckets with more than one trajectory: 3816
Buckets with only one trajectory: 12539
Largest Bucket Size: 99
Percentage of buckets with more than one trajectory: 23.33%
Percentage of buckets with only one trajectory: 76.67%


## Finds the largest bucket and outputs stats from this bucket

In [ ]:
import pandas as pd
from itertools import combinations

ROME_TRUE_SIMILARITY_FILE = "../../results_true/similarity_values/rome/dtw/rome-dtw-3050.csv"

# Load the similarity matrix CSV
similarity_df = pd.read_csv(ROME_TRUE_SIMILARITY_FILE, index_col=0)

# Results storage for statistics
bucket_stats = {}

# Filter buckets to process only those with more than one trajectory
filtered_buckets = {bucket: trajectories for bucket, trajectories in bucket_system.items() if len(trajectories) > 1}

# Track the bucket with the most trajectories
max_bucket_size = 0
max_bucket = None

# Process each filtered bucket
for bucket, trajectories in filtered_buckets.items():
    similarities = []
    best_pair = None
    worst_pair = None

    # Compute all pairwise similarities within the bucket
    for t1, t2 in combinations(trajectories, 2):
        # Strip `.txt` from the trajectory names
        t1_clean = t1.replace('.txt', '')
        t2_clean = t2.replace('.txt', '')

        # Ensure correct row-column order (t1_clean should be lexicographically larger)
        if t1_clean < t2_clean:
            t1_clean, t2_clean = t2_clean, t1_clean  # Swap for correct matrix access

        if t1_clean in similarity_df.index and t2_clean in similarity_df.columns:
            similarity = float(similarity_df.at[t1_clean, t2_clean])  # Convert to native Python float
            similarities.append(similarity)
            
            # Track the pair with the best similarity (lowest value)
            if best_pair is None or similarity < best_pair[0]:
                best_pair = (similarity, t1, t2)

            # Track the pair with the worst similarity (highest value)
            if worst_pair is None or similarity > worst_pair[0]:
                worst_pair = (similarity, t1, t2)

    if similarities:
        best_similarity = min(similarities)
        worst_similarity = max(similarities)
        avg_similarity = sum(similarities) / len(similarities)
        bucket_stats[bucket] = {
            "best": best_similarity,
            "worst": worst_similarity,
            "average": avg_similarity,
            "best_pair": best_pair,
            "worst_pair": worst_pair,
        }
        
        # Check if this bucket has the most trajectories
        if len(trajectories) > max_bucket_size:
            max_bucket_size = len(trajectories)
            max_bucket = bucket
    else:
        # Handle missing trajectory pairs in the CSV
        bucket_stats[bucket] = {"best": None, "worst": None, "average": None}

# Display results for the bucket with the most trajectories
if max_bucket:
    print(f"Bucket with the most trajectories: {max_bucket} ({max_bucket_size} trajectories)")
    best_similarity, best_t1, best_t2 = bucket_stats[max_bucket]["best_pair"]
    worst_similarity, worst_t1, worst_t2 = bucket_stats[max_bucket]["worst_pair"]
    print(f"Most Similar Pair (Lowest similarity value): {best_similarity} between {best_t1} and {best_t2}")
    print(f"Least Similar Pair (Highest similarity value): {worst_similarity} between {worst_t1} and {worst_t2}")
    print(f"Average Similarity inside bucket '{max_bucket}': {bucket_stats[max_bucket]['average']:.2f}")
else:
    print("No buckets with more than one trajectory found.")



In [ ]:
print(bucket_system[83955211460528069735602233183555021590])